In [1]:
import numpy as np
import cv2
import os
import pickle
from jetcam.utils import bgr8_to_jpeg
import ipywidgets
from jetracer.nvidia_racecar import NvidiaRacecar
from IPython.display import clear_output
from time import sleep
from jetcam.csi_camera import CSICamera

In [2]:
def load_pickle(cal_dir='cal_pickle.p'):
    global mtx
    global dist
    
    with open(cal_dir, mode='rb') as f:
        file = pickle.load(f)
    mtx = file['mtx']
    dist = file['dist']
    print(mtx, dist)
    
def undistort(img):

    dst = cv2.undistort(img, mtx, dist, None, mtx)    
    return dst
    
def filterByColor(image):
    #image = cv2.equalizeHist(image)
    #_, mask = cv2.threshold(image.copy(),0,255,cv2.THRESH_OTSU)
    lower = np.array([110, 150, 110])
    upper = np.array([255, 255, 255])
    mask = cv2.inRange(image, lower, upper)
    return mask

def parse_steering(x, mid, scale, offset, x_slc, x_size, bools):
    #unpack
    l, r, m = bools
    
    #parse x sl win offset
    x_move = x[-1]-x[0]
    xm_steer = (x_move/x_size)*0.75
    
    print("xm", xm_steer)
    
    #parse offset from middle

    if l:
        lane_mid = np.median(x)+offset
        steer = (((mid-lane_mid)/mid)*0.65)/2
        print("lml",lane_mid)
        print("stl", steer)
    elif r:
        lane_mid = (x_slc+np.median(x))-offset
        steer = (((mid-lane_mid)/mid)*0.65)/2
        print("lmr",lane_mid)
        print("str", steer)

        
        
    return xm_steer+steer
    

def line_check(img, min_height = 120, max_width = 100):
    """
        img_l -> left half of image mask
        img_r -> right half of image mask
        min_height -> minimum line height
        max_width -> maximum line width
        top_slice -> num of pixels from top not to consider
    """
    #hitograms by y axis (get lane width)
    h_hist = np.sum(img, axis=1)
    w_hist = np.sum(img, axis=0)
    #print(h_hist, w_hist)
    #minimum width
    h_thr = 255*10
    w_thr = 20*255
    
    height = 0
    width = 0

    y=[]
    for i in range(len(h_hist)):
        if h_hist[i]>h_thr:
            height+=1
            y.append(i)
            
    for i in w_hist:
        if i>w_thr:
            width+=1
            
    print("H:",height, "W:", width)
            
    #h/w bools
    lane = True if height>min_height else False
    lane = lane if width<max_width else False
    #lane = False if height==0 or width == 0 else lane
    
    if len(y)>5: 
        y1 = np.min(y)
        y2 = np.max(y)
    else:
        y1 = 0
        y2 = 100
    
    return (lane, y1, y2, height)

def mid_correction(img, nwindows=9, margin=150, minpix = 1, draw_windows=True, offset=0):
    """
    offset -> mask offset from x=0
    """
    #calculate global histogram

    histogram_y = np.sum(img, axis=1)
    
    h_thr = 255/10   #white = 255, 10px
    
    y_coords = []
    for i in range(len(histogram_y)):
        if histogram_y[i]>h_thr:
            y_coords.append(i)
    
    histogram = np.sum(img[:, np.min(y_coords):np.max(y_coords)], axis=0)
    #first window starting point    
    base = np.argmax(histogram[:120])
    x_current = base
    
    window_height = np.int(img.shape[0]/nwindows)
    out_img = np.dstack((img, img, img))*255
    
    #nz
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Create empty lists to receive left and right lane pixel indices
    x_coords = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        win_x_low = x_current - margin
        win_x_high = x_current + margin
        
        # Draw the windows on the visualization image
        if draw_windows == True:
            cv2.rectangle(out_img,(win_x_low,win_y_low),(win_x_high,win_y_high),
            (100,255,255), 3)
        
        x = (win_x_low+win_x_high)/2
        x_coords.append(x+offset)        

        #next window starting position
        next_hist = np.sum(img[(win_y_low-window_height):(win_y_low), int(win_x_low-margin):int(win_x_high+margin)], axis=0)
        #print(next_hist)
        if len(next_hist) > 0:
            hig = int(win_x_low-margin)+np.argmax(next_hist)
            x_current = hig
    

def sliding_one(img, nwindows=9, margin=150, minpix = 1, draw_windows=True, offset=0):
    """
    offset -> mask offset from x=0
    """

    #calculate global histogram
    window_height = np.int(img.shape[0]/nwindows)
    histogram = np.sum(img[int(img.shape[0]-window_height):img.shape[1], :], axis=0)
    #first window starting point
    base = np.argmax(histogram)
    x_current = base
    
    out_img = np.dstack((img, img, img))*255
    #nz
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Create empty lists to receive left and right lane pixel indices
    x_coords = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        win_x_low = x_current - margin
        win_x_high = x_current + margin
        
        # Draw the windows on the visualization image
        if draw_windows == True:
            cv2.rectangle(out_img,(win_x_low,win_y_low),(win_x_high,win_y_high),
            (100,255,255), 3)
        
        x = (win_x_low+win_x_high)/2
        x_coords.append(x+offset)        

        #next window starting position
        
        next_x = int(win_x_low-margin)
        x_l = 0 if int(win_x_low-margin) < 0 else int(win_x_low-margin)
        x_h = img.shape[1] if int(win_x_high+margin) > img.shape[1] else int(win_x_high+margin)
        next_hist = np.sum(img[(win_y_low-window_height):(win_y_low), :], axis=0)
        
        if len(next_hist) > 0:
            if np.max(next_hist) >10:
                hig = np.argmax(next_hist)
                x = hig if int(win_x_low-margin) < hig else int(win_x_low-margin)
                x = x if int(win_x_high+margin) > x else int(win_x_high+margin)
                x_current = x

    return out_img, x_coords



In [3]:
#prefs
show_widgets = True   #show lanes in widgets

#steering
l_steer = 0       #last steer value
road_width = 300
offset = 35     #offset from l/r line
xl = []
xr = []
xm = []

#image undistort
mtx = list()
dist = list()
load_pickle()

#car object
car = NvidiaRacecar()

#start camera
camera = CSICamera(width=1280, height=720)
camera.running = True


#frame size
width  = 398
height = 224
mid = 199   #picture midpoint
mid_offset = 40

#perspective transform
#src=np.float32([[91, 51], [0,105], [363, 105], [277, 51]])
#dst=np.float32([[0,0],[0,224],[398, 224],[398,0]])

#src=np.float32([[73, 71], [0,119], [364, 122], [292, 71]])
#dst=np.float32([[0,0],[0,224],[398, 224],[398,0]])

src=np.float32([[68, 90], [0,145], [398, 145], [316, 90]])
dst=np.float32([[0,0],[0,224],[398, 224],[398,0]])



M = cv2.getPerspectiveTransform(src, dst)
M_inv = cv2.getPerspectiveTransform(dst, src)

#widgets
if show_widgets:
    camera_widget = ipywidgets.Image(width=width, height=height)
    pers_wid = ipywidgets.Image(width=width, height=height)
    bin_wid = ipywidgets.Image(width=width, height=height)
    line_wid = ipywidgets.Image(width=width, height=height)
    
    data_collection_widget = ipywidgets.VBox([
        ipywidgets.HBox([camera_widget, pers_wid, bin_wid, line_wid])
    ])
    display(data_collection_widget)
sleep(5)

[[1.96240521e+04 0.00000000e+00 7.12836065e+02]
 [0.00000000e+00 1.11197536e+04 3.69724434e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] [[-1.53805492e+02  2.25637670e+04 -4.53614331e-01 -1.60457524e-01
   1.23572674e+04]]


In [6]:
#car.throttle = -0.0

while True:
    l, r, m  = (False, False, False)
    frame = camera.value        
    frame = undistort(frame.copy())
    frame = cv2.resize(frame, (width, height))
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    warp = cv2.warpPerspective(frame.copy(), M, (width,height), flags=cv2.INTER_LINEAR)
    cv2.imwrite("warp_zavoj.jpg", frame)
    blurred = cv2.medianBlur(warp, 5)

    #lane find
    mask = filterByColor(blurred)
    left_half = blurred[:, :int(mid-mid_offset)] 
    mask_left = filterByColor(left_half)    
    l, y1r, y2r, hl = line_check(mask_left, min_height=22, max_width=130)
    right_half = blurred[:, int(mid+mid_offset):]
    mask_right = filterByColor(right_half)    
    r, y1l, y2l, hr = line_check(mask_right, min_height=30, max_width=130)
    
    if l and hl>hr:
        out_img, x = sliding_one(mask_left[y1r:y2r, :], nwindows=4, margin=22, minpix = 1, draw_windows=show_widgets)
    else:
        
        if r:
            out_img, x = sliding_one(mask_right[y1r:y2r, :], nwindows=4, margin=22, minpix = 1, draw_windows=show_widgets, offset=int(mid+mid_offset))
        else:
            mid_part = blurred[:, int(mid/2):int(mid+(mid/2))]
            mask_mid = filterByColor(mid_part)
            m, y1, y2, mbool = line_check(mask_mid, min_height=70, max_width=120)
            m=False
            if m:
                out_img, x = mid_correction(mask_mid, nwindows=4, margin=22, minpix = 1, draw_windows=show_widgets)
            else:
                out_img = blurred
    print(l, r, m)
        
    #parse steering
    if l and hl>hr: 
        steer = parse_steering(x, mid, mid/(y1l-y2l), 161, 0, int(mid-mid_offset), (l, False, False))
        l_steer = steer
    elif r:
        steer = parse_steering(x, mid, mid/(y1r-y2r), 399, int(mid+mid_offset), int(width-(mid+mid_offset)), (False, r, False))
        l_steer = steer
    else:
        steer = l_steer
        
    print(steer)
    print("yl", y1l, y2l)
    print("yl", y1r, y2r)
    
    car.steering = steer
    
    
    #show widgets
    if show_widgets:
        camera_widget.value = bgr8_to_jpeg(frame)
        pers_wid.value = bgr8_to_jpeg(mask)
        line_wid.value = bgr8_to_jpeg(mask_right)
        bin_wid.value = bgr8_to_jpeg(mask_left)
    #clear_output(True)

H: 212 W: 114
H: 0 W: 0
True False False
xm 0.6226415094339623
lml 227.0
stl -0.0457286432160804
0.576912866217882
yl 0 100
yl 2 223
H: 205 W: 114
H: 0 W: 0
True False False
xm 0.6226415094339623
lml 227.0
stl -0.0457286432160804
0.576912866217882
yl 0 100
yl 2 223
H: 206 W: 114
H: 0 W: 0
True False False
xm 0.6226415094339623
lml 227.0
stl -0.0457286432160804
0.576912866217882
yl 0 100
yl 3 220
H: 207 W: 113
H: 0 W: 0
True False False
xm 0.6226415094339623
lml 227.0
stl -0.0457286432160804
0.576912866217882
yl 0 100
yl 3 221
H: 207 W: 113
H: 0 W: 0
True False False
xm 0.6226415094339623
lml 227.0
stl -0.0457286432160804
0.576912866217882
yl 0 100
yl 4 222
H: 210 W: 113
H: 0 W: 0
True False False
xm 0.6226415094339623
lml 227.0
stl -0.0457286432160804
0.576912866217882
yl 0 100
yl 3 221
H: 216 W: 112
H: 0 W: 0
True False False
xm 0.6226415094339623
lml 227.0
stl -0.0457286432160804
0.576912866217882
yl 0 100
yl 3 223
H: 213 W: 114
H: 0 W: 0
True False False
xm 0.6226415094339623
lml 22

KeyboardInterrupt: 